<a href="https://colab.research.google.com/github/KarelGans/Playstore-Comment-Sentiment-Analysis/blob/main/Proyek_Analisis_Sentimen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

## Import Library

In [1]:
# Install required packages (run once)
!pip install google-play-scraper numpy==1.23.5 gensim sastrawi wordcloud nltk scikit-learn

# Import required libraries
import pandas as pd
import numpy as np
import datetime as dt
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud
from google_play_scraper import app, reviews, Sort, reviews_all
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec
import nltk
import requests
from io import StringIO
import csv
import joblib

# Download NLTK resources
nltk.download('punkt_tab')
nltk.download('stopwords')  # Only download once to save time

# Set random seed for reproducibility
seed = 50
np.random.seed(seed)

# Disable chained assignment warning
pd.options.mode.chained_assignment = None


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Load Dataset

In [2]:
# Membuat DataFrame dari hasil scrapreview
app_reviews_df = pd.read_csv('ulasan_aplikasi.csv', encoding='utf-8')

# Menghitung jumlah baris dan kolom dalam DataFrame
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape

# Menampilkan lima baris pertama dari DataFrame app_reviews_df
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,471db5f2-8dc4-4d12-968c-8e8be99c1997,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Sudah lama pake aplikasi ini Alhamdulillah san...,3,6,4.6.0,2025-03-18 10:04:46,Mohon maaf atas kendala yang Bapak/Ibu alami. ...,2025-03-18 10:39:05,4.6.0
1,73e00a87-fef3-4680-afa6-ab7daef9fd80,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"apknya udah bagus, tapi menurutku ada beberapa...",4,2,4.6.1,2025-03-21 19:01:42,Mohon maaf atas ketidaknyamanannya. Saran dan ...,2025-03-21 19:11:52,4.6.1
2,56fd0f64-599d-46a2-a862-e3c6d4a3f146,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sering tiba² harus verifikasi lagi, setelah pr...",3,18,4.6.0,2025-03-13 21:12:09,Mohon maaf atas ketidaknyamanannya. Saran dan ...,2025-03-13 22:05:47,4.6.0
3,c98d5b73-54c4-4fea-aa9f-b897c9e3493d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Udah bener bener kemarin ga perlu nonaktifkan ...,1,31,4.6.0,2025-03-14 01:54:22,Mhn maaf atas ketidaknyamanannya. Saat ini tdk...,2025-03-14 08:52:53,4.6.0
4,37226f0e-d16a-43b4-9d4b-72a91fd0f853,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Habis stabil aplikasinya beberapa bulan, resek...",1,19,4.6.0,2025-03-12 11:24:49,"Mohon maaf atas ketidaknyamanan Bapak/Ibu, unt...",2025-03-12 11:57:33,4.6.0


In [3]:
# Menampilkan informasi tentang DataFrame app_reviews_df
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112500 entries, 0 to 112499
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   reviewId              112500 non-null  object
 1   userName              112500 non-null  object
 2   userImage             112500 non-null  object
 3   content               112500 non-null  object
 4   score                 112500 non-null  int64 
 5   thumbsUpCount         112500 non-null  int64 
 6   reviewCreatedVersion  92556 non-null   object
 7   at                    112500 non-null  object
 8   replyContent          107875 non-null  object
 9   repliedAt             107875 non-null  object
 10  appVersion            92556 non-null   object
dtypes: int64(2), object(9)
memory usage: 9.4+ MB


In [4]:
# Membuat DataFrame baru (clean_df) dengan menghapus baris yang memiliki nilai yang hilang (NaN) dari app_reviews_df
clean_df = app_reviews_df.dropna()

In [5]:
# Menghapus baris duplikat dari DataFrame clean_df
clean_df = clean_df.drop_duplicates()

# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape

## Text Preprocessing

In [6]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [7]:
def load_slang_dict(filename):
    """
    Converts a tab-separated text file of slang-formal word pairs into a Python dictionary

    Args:
        filename (str): Path to the text file

    Returns:
        dict: Dictionary with slang terms as keys and formal equivalents as values
    """
    slang_dict = {}

    with open(filename, 'r') as file:
        for line in file:
            # Remove whitespace and skip empty lines
            line = line.strip()
            if not line:
                continue

            # Split on tab character
            parts = line.split('\t')

            # Ensure we have exactly 2 elements (slang and formal)
            if len(parts) == 2:
                slang, formal = parts
                slang_dict[slang] = formal

    return slang_dict

slangwords = load_slang_dict("kbba.txt")
print(slangwords)

{'7an': 'tujuan', '@': 'di', 'ababil': 'abg labil', 'abis': 'habis', 'acc': 'accord', 'ad': 'ada', 'adlah': 'adalah', 'adlh': 'adalah', 'adoh': 'aduh', 'afaik': 'as far as i know', 'aha': 'tertawa', 'ahaha': 'haha', 'aing': 'saya', 'aj': 'saja', 'aja': 'saja', 'ajep-ajep': 'dunia gemerlap', 'ajj': 'saja', 'ak': 'saya', 'aka': 'dikenal juga sebagai', 'akika': 'aku', 'akko': 'aku', 'akkoh': 'aku', 'akku': 'aku', 'akuwh': 'aku', 'akyu': 'aku', 'alay': 'norak', 'aljasa': 'asal jadi saja', 'alow': 'halo', 'ama': 'sama', 'ambilin': 'ambilkan', 'ambl': 'ambil', 'ancur': 'hancur', 'anjir': 'anjing', 'anjrit': 'anjing', 'ank': 'anak', 'anter': 'antar', 'ap': 'apa', 'ap2': 'apa-apa', 'apaan': 'apa', 'apasih': 'apa sih', 'ape': 'apa', 'apes': 'sial', 'aplot': 'unggah', 'aps': 'apa', 'apva': 'apa', 'aq': 'saya', 'aqu': 'aku', 'aquwh': 'aku', 'asap': 'sesegera mungkin', 'asbun': 'asal bunyi', 'aseek': 'asyik', 'aseekk': 'asyik', 'asek': 'asyik', 'asekk': 'asyik', 'aseknya': 'asyiknya', 'asem': 'asa

In [8]:
#slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"} #perlu cari dict slang karena dicoding kasih seuprit
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [9]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)



In [10]:
# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

In [11]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,471db5f2-8dc4-4d12-968c-8e8be99c1997,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Sudah lama pake aplikasi ini Alhamdulillah san...,3,6,4.6.0,2025-03-18 10:04:46,Mohon maaf atas kendala yang Bapak/Ibu alami. ...,2025-03-18 10:39:05,4.6.0,Sudah lama pake aplikasi ini Alhamdulillah san...,sudah lama pake aplikasi ini alhamdulillah san...,sudah lama pakai aplikasi ini alhamdulillah sa...,"[sudah, lama, pakai, aplikasi, ini, alhamdulil...","[pakai, aplikasi, alhamdulillah, membantu, kec...",pakai aplikasi alhamdulillah membantu kecewa t...
1,73e00a87-fef3-4680-afa6-ab7daef9fd80,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"apknya udah bagus, tapi menurutku ada beberapa...",4,2,4.6.1,2025-03-21 19:01:42,Mohon maaf atas ketidaknyamanannya. Saran dan ...,2025-03-21 19:11:52,4.6.1,apknya udah bagus tapi menurutku ada beberapa ...,apknya udah bagus tapi menurutku ada beberapa ...,apknya sudah bagus tapi menurutku ada beberapa...,"[apknya, sudah, bagus, tapi, menurutku, ada, b...","[apknya, bagus, menurutku, hall, ganti, kayak,...",apknya bagus menurutku hall ganti kayak warnan...
2,56fd0f64-599d-46a2-a862-e3c6d4a3f146,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sering tiba² harus verifikasi lagi, setelah pr...",3,18,4.6.0,2025-03-13 21:12:09,Mohon maaf atas ketidaknyamanannya. Saran dan ...,2025-03-13 22:05:47,4.6.0,Sering tiba² harus verifikasi lagi setelah pro...,sering tiba² harus verifikasi lagi setelah pro...,sering tiba² harus verifikasi lagi setelah pro...,"[sering, tiba², harus, verifikasi, lagi, setel...","[tiba², verifikasi, proses, verifikasi, sms, m...",tiba² verifikasi proses verifikasi sms mengisi...
3,c98d5b73-54c4-4fea-aa9f-b897c9e3493d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Udah bener bener kemarin ga perlu nonaktifkan ...,1,31,4.6.0,2025-03-14 01:54:22,Mhn maaf atas ketidaknyamanannya. Saat ini tdk...,2025-03-14 08:52:53,4.6.0,Udah bener bener kemarin ga perlu nonaktifkan ...,udah bener bener kemarin ga perlu nonaktifkan ...,sudah benar benar kemarin tidak perlu nonaktif...,"[sudah, benar, benar, kemarin, tidak, perlu, n...","[kemarin, nonaktifkan, aksebilitas, eh, update...",kemarin nonaktifkan aksebilitas eh update nona...
4,37226f0e-d16a-43b4-9d4b-72a91fd0f853,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Habis stabil aplikasinya beberapa bulan, resek...",1,19,4.6.0,2025-03-12 11:24:49,"Mohon maaf atas ketidaknyamanan Bapak/Ibu, unt...",2025-03-12 11:57:33,4.6.0,Habis stabil aplikasinya beberapa bulan resek ...,habis stabil aplikasinya beberapa bulan resek ...,habis stabil aplikasinya beberapa bulan resek ...,"[habis, stabil, aplikasinya, beberapa, bulan, ...","[habis, stabil, aplikasinya, resek, aksesibili...",habis stabil aplikasinya resek aksesibilitas m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112495,1991d700-7c60-4776-b10a-7f693e794307,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Pelayananya buruk lola sya dari jam 11 sampai ...,1,0,2.7.1,2021-02-10 17:20:14,"Mohon maaf atas ketidaknyamanannya Bapak/Ibu, ...",2021-02-11 08:53:11,2.7.1,Pelayananya buruk lola sya dari jam sampai g...,pelayananya buruk lola sya dari jam sampai g...,pelayananya buruk lambat berfikir sya dari jam...,"[pelayananya, buruk, lambat, berfikir, sya, da...","[pelayananya, buruk, lambat, berfikir, sya, ja...",pelayananya buruk lambat berfikir sya jam nyam...
112496,3f5e0a3a-8683-4139-8b61-451d4c1b83f1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,BCA mobile sangat membantu dalam kegiatan bisn...,5,0,2.9.5,2022-09-01 20:51:28,Terima kasih atas ulasannya. Semoga aplikasi B...,2022-09-01 21:59:16,2.9.5,BCA mobile sangat membantu dalam kegiatan bisn...,

## Pelabelan

In [12]:
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [13]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score >= 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif

    # else:
    #     polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [14]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
positive    48391
negative    40140
Name: count, dtype: int64


## Data Splitting, Feature Extraction and Modeling

In [15]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity
0,471db5f2-8dc4-4d12-968c-8e8be99c1997,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Sudah lama pake aplikasi ini Alhamdulillah san...,3,6,4.6.0,2025-03-18 10:04:46,Mohon maaf atas kendala yang Bapak/Ibu alami. ...,2025-03-18 10:39:05,4.6.0,Sudah lama pake aplikasi ini Alhamdulillah san...,sudah lama pake aplikasi ini alhamdulillah san...,sudah lama pakai aplikasi ini alhamdulillah sa...,"[sudah, lama, pakai, aplikasi, ini, alhamdulil...","[pakai, aplikasi, alhamdulillah, membantu, kec...",pakai aplikasi alhamdulillah membantu kecewa t...,6,positive
1,73e00a87-fef3-4680-afa6-ab7daef9fd80,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"apknya udah bagus, tapi menurutku ada beberapa...",4,2,4.6.1,2025-03-21 19:01:42,Mohon maaf atas ketidaknyamanannya. Saran dan ...,2025-03-21 19:11:52,4.6.1,apknya udah bagus tapi menurutku ada beberapa ...,apknya udah bagus tapi menurutku ada beberapa ...,apknya sudah bagus tapi menurutku ada beberapa...,"[apknya, sudah, bagus, tapi, menurutku, ada, b...","[apknya, bagus, menurutku, hall, ganti, kayak,...",apknya bagus menurutku hall ganti kayak warnan...,-13,negative
2,56fd0f64-599d-46a2-a862-e3c6d4a3f146,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sering tiba² harus verifikasi lagi, setelah pr...",3,18,4.6.0,2025-03-13 21:12:09,Mohon maaf atas ketidaknyamanannya. Saran dan ...,2025-03-13 22:05:47,4.6.0,Sering tiba² harus verifikasi lagi setelah pro...,sering tiba² harus verifikasi lagi setelah pro...,sering tiba² harus verifikasi lagi setelah pro...,"[sering, tiba², harus, verifikasi, lagi, setel...","[tiba², verifikasi, proses, verifikasi, sms, m...",tiba² verifikasi proses verifikasi sms mengisi...,27,positive
3,c98d5b73-54c4-4fea-aa9f-b897c9e3493d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Udah bener bener kemarin ga perlu nonaktifkan ...,1,31,4.6.0,2025-03-14 01:54:22,Mhn maaf atas ketidaknyamanannya. Saat ini tdk...,2025-03-14 08:52:53,4.6.0,Udah bener bener kemarin ga perlu nonaktifkan ...,udah bener bener kemarin ga perlu nonaktifkan ...,sudah benar benar kemarin tidak perlu nonaktif...,"[sudah, benar, benar, kemarin, tidak, perlu, n...","[kemarin, nonaktifkan, aksebilitas, eh, update...",kemarin nonaktifkan aksebilitas eh update nona...,10,positive
4,37226f0e-d16a-43b4-9d4b-72a91fd0f853,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Habis stabil aplikasinya beberapa bulan, resek...",1,19,4.6.0,2025-03-12 11:24:49,"Mohon maaf atas ketidaknyamanan Bapak/Ibu, unt...",2025-03-12 11:57:33,4.6.0,Habis stabil aplikasinya beberapa bulan resek ...,habis stabil aplikasinya beberapa bulan resek ...,habis stabil aplikasinya beberapa bulan resek ...,"[habis, stabil, aplikasinya, beberapa, bulan, ...","[habis, stabil, aplikasinya, resek, aksesibili...",habis stabil aplikasinya resek aksesibilitas m...,-26,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112495,1991d700-7c60-4776-b10a-7f693e794307,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Pelayananya buruk lola sya dari jam 11 sampai ...,1,0,2.7.1,2021-02-10 17:20:14,"Mohon maaf atas ketidaknyamanannya Bapak/Ibu, ...",2021-02-11 08:53:11,2.7.1,Pelayananya buruk lola sya dari jam sampai g...,pelayananya buruk lola sya dari jam sampai g...,pelayananya buruk lambat berfikir sya dari jam...,"[pelayananya, buruk, lambat, berfikir, sya, da...","[pelayananya, buruk, lambat, berfikir, sya, ja...",pelayananya buruk lambat berfikir sya jam nyam...,-4,negative
112496,3f5e0a3a-8683-4139-8b61-451d4c1b83f1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,BCA mobile sangat membantu dalam kegiatan bisn...,5,0,2.9.5,2022-09-01 20:51:28,Terima kasih atas ula

In [16]:
X = clean_df['text_akhir']
y = clean_df['polarity']

In [22]:
# ---- SVM with TF-IDF (70/30 Split) ----
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8)
X_tfidf = tfidf.fit_transform(X)
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

svm_model = SVC()
svm_model.fit(X_train_tfidf, y_train_tfidf)

# Accuracy for SVM with TF-IDF
y_pred_train_svm = svm_model.predict(X_train_tfidf)
y_pred_test_svm = svm_model.predict(X_test_tfidf)
accuracy_train_svm = accuracy_score(y_train_tfidf, y_pred_train_svm)
accuracy_test_svm = accuracy_score(y_test_tfidf, y_pred_test_svm)

print("SVM with TF-IDF - Accuracy Train:", accuracy_train_svm)
print("SVM with TF-IDF - Accuracy Test:", accuracy_test_svm)


SVM with TF-IDF - Accuracy Train: 0.9212534895354278
SVM with TF-IDF - Accuracy Test: 0.8886295180722892


In [23]:
# ---- RF with TF-IDF (70/30 Split) ----
X_train_tfidf_70, X_test_tfidf_30, y_train_tfidf_70, y_test_tfidf_30 = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

rf_tfidf_model = RandomForestClassifier(n_estimators=150, random_state=42)
rf_tfidf_model.fit(X_train_tfidf_70, y_train_tfidf_70)

# Accuracy for RF with TF-IDF
y_pred_train_rf_tfidf = rf_tfidf_model.predict(X_train_tfidf_70)
y_pred_test_rf_tfidf = rf_tfidf_model.predict(X_test_tfidf_30)
accuracy_train_rf_tfidf = accuracy_score(y_train_tfidf_70, y_pred_train_rf_tfidf)
accuracy_test_rf_tfidf = accuracy_score(y_test_tfidf_30, y_pred_test_rf_tfidf)

print("RF with TF-IDF - Accuracy Train:", accuracy_train_rf_tfidf)
print("RF with TF-IDF - Accuracy Test:", accuracy_test_rf_tfidf)


RF with TF-IDF - Accuracy Train: 0.9746494327992126
RF with TF-IDF - Accuracy Test: 0.8657379518072289


In [26]:
# ---- Logistiv with Word2Vec (80/20 Split) ----
X_tokenized = [text.split() for text in X]
w2v_model = Word2Vec(sentences=X_tokenized, vector_size=600, window=15, min_count=2, workers=4, sg=1)

def get_word2vec_features(texts, model, vector_size=600): # Changed vector_size to 500
    features = []
    for text in texts:
        vectors = [model.wv[word] for word in text if word in model.wv]
        if vectors:
            features.append(np.mean(vectors, axis=0))
        else:
            features.append(np.zeros(vector_size)) # Now using correct vector_size
    return np.array(features)

X_w2v = get_word2vec_features(X_tokenized, w2v_model, vector_size=600)
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y, test_size=0.2, random_state=42)

from sklearn.linear_model import LogisticRegression
logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X_train_w2v, y_train_w2v)

# Accuracy for Logistic Regression with W2V
y_pred_train_logreg = logreg_model.predict(X_train_w2v)
y_pred_test_logreg = logreg_model.predict(X_test_w2v)
accuracy_train_logreg = accuracy_score(y_train_w2v, y_pred_train_logreg)
accuracy_test_logreg = accuracy_score(y_test_w2v, y_pred_test_logreg)

print("Logistic Regression with W2V - Accuracy Train:", accuracy_train_logreg)
print("Logistic Regression with W2V - Accuracy Test:", accuracy_test_logreg)

Logistic Regression with W2V - Accuracy Train: 0.8400542189088445
Logistic Regression with W2V - Accuracy Test: 0.8362229626701304


In [27]:
# ---- Logistic Regression with TF-IDF (80/20 Split) ----
tfidf = TfidfVectorizer(max_features=500, min_df=17, max_df=0.8)
X_tfidf = tfidf.fit_transform(X)
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(X_train_tfidf, y_train_tfidf)

# Accuracy for Logistic Regression with TF-IDF
y_pred_train_logreg = logreg_model.predict(X_train_tfidf)
y_pred_test_logreg = logreg_model.predict(X_test_tfidf)
accuracy_train_logreg = accuracy_score(y_train_tfidf, y_pred_train_logreg)
accuracy_test_logreg = accuracy_score(y_test_tfidf, y_pred_test_logreg)

print("Logistic Regression with TF-IDF - Accuracy Train:", accuracy_train_logreg)
print("Logistic Regression with TF-IDF - Accuracy Test:", accuracy_test_logreg)


Logistic Regression with TF-IDF - Accuracy Train: 0.9323393200045182
Logistic Regression with TF-IDF - Accuracy Test: 0.9270909809679788


### Cell Inteference untuk model logistic - TF-IDF - 80/20

In [28]:
# Save the model and vectorizer for later use
joblib.dump(logreg_model, "logistic_regression_model.pkl")
joblib.dump(tfidf, "tfidf_vectorizer.pkl")

# Function to perform inference
def predict_sentiment(text):
    """
    Predicts the sentiment (Negatif, Netral, Positif) for a given text.
    """
    # Load the trained model and vectorizer
    model = joblib.load("logistic_regression_model.pkl")
    vectorizer = joblib.load("tfidf_vectorizer.pkl")

    # Transform input text into TF-IDF features
    text_tfidf = vectorizer.transform([text])

    # Predict class label
    predicted_class = model.predict(text_tfidf)[0]

    return predicted_class

# contoh dengan data baru
test_texts = ["BCA mobile sangat membantu saya berjualan", "Saya merasa cukup baik saja", "Layanan sangat buruk", "jelek UI aplikasinya. Tolong perbaiki"]
for text in test_texts:
    print(f"Input: {text} -> Predicted Sentiment: {predict_sentiment(text)}")


Input: BCA mobile sangat membantu saya berjualan -> Predicted Sentiment: positive
Input: Saya merasa cukup baik saja -> Predicted Sentiment: positive
Input: Layanan sangat buruk -> Predicted Sentiment: negative
Input: jelek UI aplikasinya. Tolong perbaiki -> Predicted Sentiment: negative
